In [11]:
from src.Creditcardfaultdetection.logger import logging
logging.info('Importing Libraries')
import pandas as pd

In [12]:
import os

In [5]:
os.getcwd()

'd:\\ML\\Credit card fraud detection\\notebooks'

In [8]:
os.makedirs('name', exist_ok=True)

In [9]:
class DataIngestionConfig:
    # Paths for data storage
    raw_data_path: str = os.path.join("artifacts", "raw.csv")
    train_data_path: str = os.path.join("artifacts", "train.csv")
    test_data_path: str = os.path.join("artifacts", "test.csv")


In [10]:
class DataIngestion:
    def __init__(self):
        self.ingestion_config = DataIngestionConfig()
    def initiate_data_ingestion(self):
        logging.info("Data ingestion process started.")
        data_path=os.path.join("data","raw.csv")
        data = pd.read_csv(data_path)
        logging.info(f"Dataset loaded successfully with shape: {data.shape}")

            # Creating directory if it does not exist
        os.makedirs(os.path.dirname(self.ingestion_config.raw_data_path), exist_ok=True)

            # Saving raw data
        data.to_csv(self.ingestion_config.raw_data_path, index=False)

In [24]:
from pathlib import Path
import pandas as pd

file_path = Path("data") / "Credit_Card.csv"

data=pd.read_csv(file_path)
os.makedirs(os.path.dirname("artifacts/raw.csv"), exist_ok=True)
data.to_csv("artifacts/raw.csv", index=False)

In [20]:
os.getcwd()

'd:\\ML\\Credit card fraud detection\\notebooks'